# Ion–particle coagulation

The problem here: positive particles (1 to 9 charges) of radius 3 nm colliding with a negative ion of radius 0.45 nm.

Several coagulation approximations:
- hard shpere
- gh2012: https://journals.aps.org/pre/abstract/10.1103/PhysRevE.85.026410
- gk2008: https://journals.aps.org/pre/abstract/10.1103/PhysRevE.78.046402
- dy2007: https://aip.scitation.org/doi/10.1063/1.2713719
- cg2019: https://www.tandfonline.com/doi/suppl/10.1080/02786826.2019.1614522
- lf2013: https://www.tandfonline.com/doi/full/10.1080/02786826.2013.783684

In [1]:
import numpy as np
from matplotlib import pyplot as plt
# from particula.util.lf2013_coagulation import lf2013_coag_full
import particula as par

## Setup

This block is setting up the properties of a set of aerosol particles, calculating key parameters related to their movement and interaction within a gaseous environment, and determining their diffusion behavior.

### **1. Environmental Setup**
- The temperature is set to **278.0 K**, which will be used for further calculations.

### **2. Particle Properties**
- **Particle Radius:** A set of particles is defined with radii ranging from **0.45 nm to 3 nm**.
- **Charge:** The particles have a range of electric charges from **-1 to +12**.
- **Density:** The particle density is primarily set to **1700 kg/m³**, with one particle (ion) at **1860 kg/m³**.

### **3. Mass Calculation**
- Using the formula for the volume of a sphere and the given densities, the **mass of each particle** is computed.

### **4. Gas Properties**
- The **dynamic viscosity** of the surrounding gas is obtained using the `par.gas.get_dynamic_viscosity()` function.

### **5. Knudsen Number Calculation**
- The **Knudsen number**, which determines whether the particle behavior is in the free molecular, transition, or continuum regime, is computed using:
  - The **mean free path** of gas molecules (from `par.gas.get_molecule_mean_free_path()`).
  - The particle radius.

### **6. Friction Factor**
- The **friction factor**, which influences the motion and diffusion of the particles, is calculated using:
  - Particle radius
  - Dynamic viscosity
  - Slip correction factor (using `par.particles.get_cunningham_slip_correction()`).

### **7. Coulomb Enhancement Ratio**
- The **Coulomb potential ratio**, which determines how electrostatic forces influence particle interactions, is computed using:
  - Particle radius
  - Charge
  - Temperature

### **8. Diffusive Knudsen Number**
- Finally, the **diffusive Knudsen number**, which governs diffusion behavior, is computed using:
  - Particle radius
  - Particle mass
  - Friction factor
  - Coulomb potential ratio
  - Temperature


In [ ]:
# environment
temperature = 278.0
pressure = 101325.0

# get each particle properties
particle_radius = np.array([0.45e-9, 3e-9, 3e-9, 3e-9, 3e-9, 3e-9, 3e-9, 3e-9, 3e-9, 3e-9, 3e-9, 3e-9, 3e-9])
charge = np.array([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
particle_density = np.array([1860, 1700, 1700, 1700, 1700, 1700, 1700, 1700, 1700, 1700, 1700, 1700, 1700])

# calculate mass
particle_mass = 4/3 * np.pi * particle_radius**3 * particle_density

# get properties
dynamic_viscosity = par.gas.get_dynamic_viscosity(temperature=temperature)

# get knudsen number
knudsen_number = par.particles.get_knudsen_number(
    mean_free_path=par.gas.get_molecule_mean_free_path(temperature=temperature, dynamic_viscosity=dynamic_viscosity, pressure=pressure),
    particle_radius=particle_radius
)
# get friction factor
friction_factor = par.particles.get_friction_factor(
    particle_radius=particle_radius,
    dynamic_viscosity=par.gas.get_dynamic_viscosity(temperature=temperature),
    slip_correction=par.particles.get_cunningham_slip_correction(knudsen_number=knudsen_number),
)

# get coulomb potential ratio
coulomb_potential_ratio = par.particles.get_coulomb_enhancement_ratio(
    particle_radius=particle_radius,
    charge=charge,
    temperature=temperature,
)

# get diffusive knudsen number
diffusive_knudsen = par.particles.get_diffusive_knudsen_number(
    particle_radius=particle_radius,
    particle_mass=particle_mass,
    friction_factor=friction_factor,
    coulomb_potential_ratio=coulomb_potential_ratio,
    temperature=temperature,
)

In [11]:
# hard sphere dimensionless kernel
hard_sphere_dimensionless_kernel = par.dynamics.get_hard_sphere_kernel(
    diffusive_knudsen=diffusive_knudsen,
)

# coulomb dimensionless kernel for Dyachkov et al. (2007)
coulomb_dyachkov2007_dimensionless_kernel = par.dynamics.get_coulomb_kernel_dyachkov2007(
    diffusive_knudsen=diffusive_knudsen,
    coulomb_potential_ratio=coulomb_potential_ratio,
)

# coulomb dimensionless kernel for Gatti et al. (2008)
coulomb_gatti2008_dimensionless_kernel = par.dynamics.get_coulomb_kernel_gatti2008(
    diffusive_knudsen=diffusive_knudsen,
    coulomb_potential_ratio=coulomb_potential_ratio,
)

# coulomb dimensionless kernel for Gopalakrishnan et al. (2012)
coulomb_g2012_dimensionless_kernel = par.dynamics.get_coulomb_kernel_gopalakrishnan2012(
    diffusive_knudsen=diffusive_knudsen,
    coulomb_potential_ratio=coulomb_potential_ratio,
)

# coulomb dimensionless kernel for Chahl et al. (2019)
coulomb_chahl20019_dimensionless_kernel = par.dynamics.get_coulomb_kernel_chahl2019(
    diffusive_knudsen=diffusive_knudsen,
    coulomb_potential_ratio=coulomb_potential_ratio,
)

In [10]:
approx = ["hardsphere", "gk2008", "gh2012", "dy2007", "cg2019"]
for i in range(len(approx)):
    shared_dict = {"temperature": 278, "coagulation_approximation": approx[i]}
    positive_particles = [
        particle.Particle(
            **shared_dict,
            particle_radius=3e-9,
            particle_charge=i,
            particle_density=1700,
        )
        for i in range(1, 10)
    ]
    negative_ion = particle.Particle(
        **shared_dict,
        particle_radius=0.45e-9,
        particle_charge=-1,
        particle_density=1860,
    )
    coags = [
        negative_ion.coagulation(particle) for particle in positive_particles
    ]
    coags_ccps_mags = [coags[index].to("cc/s").m for index in range(0, 9)]
    plt.semilogy(
        range(1, 10), np.array(coags_ccps_mags).squeeze(), label=approx[i]
    )


ret = np.nan_to_num(
    lf2013_coag_full(
        ion_type="air",
        particle_type="conductive",
        temperature_val=298.15,
        pressure_val=101325,
        charge_vals=np.arange(1, 10),
        radius_vals=3e-9,
    )[0],
    0,
)  # [0] returns negative, [1] returns positive ions
plt.semilogy(range(1, 10), np.array(ret * 1e6).squeeze(), label="lf2013")
plt.ylim([1e-7, 1e-4])
plt.ylabel("Attachment coefficient, cc/s")
plt.legend()
plt.xlabel("# charges");

NameError: name 'particle' is not defined